In [2]:
import mysql.connector
import pandas as pd

In [25]:
#Coneccion con la bd
my_db = mysql.connector.connect(
    host = "bfrbtunhryqfh3eywkgx-mysql.services.clever-cloud.com", 
    user = "ub8frnffaufhoqum",
    passwd = "4QWMy9S2vwBaQgYGI3zu",
    database = "bfrbtunhryqfh3eywkgx",
)
my_cursor = my_db.cursor()

In [10]:
#importa data

data_empleos = pd.read_csv("C:\\Users\\acer\\Desktop\\Nueva carpeta\\T1_Jobs_RS\\Data\\Data_completa_20211103.csv")
lista_empresas = data_empleos.Company.unique()

data_empleos[["Academy_grade", "Years_Experience", "SoftSkill", "HardSkill"]] = data_empleos[["Academy_grade", "Years_Experience", "SoftSkill", "HardSkill"]].fillna(" ")
data_empleos[["Salary_min", "Salary_max"]] = data_empleos[["Salary_min", "Salary_max"]] .fillna(0)

In [8]:
my_cursor = my_db.cursor()
my_cursor.execute("Show tables")
for db in my_cursor:
    print(db)


('aspirantes',)
('aspirantes_empleos',)
('empleos',)
('empresa',)


In [14]:
#insert empresas
for company in lista_empresas:
    email = company.split(" ", 2)
    try:
        email = str(email[0]) + "_" + str(email[1]) + "gmail.com"
    except IndexError:
        email = str(email[0]) + "@gmail.com"
    #print(email)
    my_cursor.execute("""
    insert into empresa (ID, Nombre,correoElectronico,Contraseña, razonSocial)
    values (%s,%s,%s,%s,%s)
    """,
    (None, company,email,"password","12345" )
    )
my_db.commit()

In [17]:
my_cursor.execute("SELECT * FROM empresa")
data_empresas = pd.DataFrame(my_cursor.fetchall())
#Tomar ID y nombre
data_empresas = data_empresas.loc[:, [0,1]]
data_empresas.rename(columns={0:'id', 1:'Company'}, inplace=True)
data_empresas

,id,Company
0,1,luxoft
1,2,kavak.com
2,3,indus software solutions
3,4,zoolatech
4,5,confidencial
...,...,...
338,339,dsi
339,340,localadventures
340,341,trato
341,342,documentos digitales


In [18]:
#Añadir id de la empresa al data frame
data_empleos = pd.merge(data_empleos, data_empresas, on="Company")

In [21]:
for row in data_empleos.itertuples():
    my_cursor.execute("""
    insert into empleos (ID, Titulo,Ubicacion,Descripcion, salarioMIN, salarioMAX, tipoPuesto,Idiomas, Escolaridad, experienciaLaboral, softSkills, hardSkills,Remoto,idEmpresa)
    values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """,
    (None, row.Title ,row.Location,row.Description,row.Salary_min, row.Salary_max, row.Tipo_Puesto, row.English, row.Academy_grade, row.Years_Experience,row.SoftSkill ,row.HardSkill, row.Remoto, row.id)
    )
my_db.commit()

In [18]:
my_db.close()